In [2]:
import json
import random
with open("C:/Users/simok/Deep_learning_in_LangTech_course/data/imdb_train.json") as f:
    data=json.load(f)
random.shuffle(data)
print(data[0])


{'class': 'neg', 'text': "I've seen this film several times in a variety of short-film festivals and it always causes me the impression that i have seen a movie trailer!   For a school-film is very well produced and directed, but the story... well it needed something else to be a bigger and interesting film. The character named Tim Watcher needed some in-dept approach. This is something that lacks in some Portuguese short films - the script is always superficial.  But still... i liked this movie...  Parabens! (congratulations!)"}


In [3]:
texts=[one_example["text"] for one_example in data]
labels=[one_example["class"] for one_example in data]
print(texts[:1])
print(labels[:1])


["I've seen this film several times in a variety of short-film festivals and it always causes me the impression that i have seen a movie trailer!   For a school-film is very well produced and directed, but the story... well it needed something else to be a bigger and interesting film. The character named Tim Watcher needed some in-dept approach. This is something that lacks in some Portuguese short films - the script is always superficial.  But still... i liked this movie...  Parabens! (congratulations!)"]
['neg']


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy
analyzer=CountVectorizer(lowercase=True).build_analyzer() # includes tokenizer and preprocessing
with open("imdb_harj.txt", "wt") as f:
    for text in texts:
        words=analyzer(text)
        doc=" ".join(words)
print(doc,file=f)

ModuleNotFoundError: No module named 'scipy'

In [ ]:
#sen jälkeen terminaalissa (avaa uusi täbi, cntrl+shift t):  git clone https://github.com/tmikolov/word2vec.git
#terminaalissa word2vec sisällä: make
#./word2vec
#esimerkki: ./word2vec -train ../imdb_harj.txt -output vec.txt -size 200 -window 5 -sample 1e-4 -negative 5 -hs 0 -binary 0 -cbow 1
#sen jälkeen jupyteriin: w2v similarity kokeilut

In [ ]:
from gensim.models import KeyedVectors
model=KeyedVectors.load_word2vec_format("word2vec/vec.txt", binary=False, limit=50000)

print("Most similar words for 'funny':")
print(model.most_similar("funny",topn=10))

print("Similarity of 'bad' and 'horrible':")
print(model.similarity("bad", "horrible"))

print("Similarity of 'bad' and 'excellent':")
print(model.similarity("bad", "excellent"))

print("Similarity of 'actor' and 'actress':")
print(model.similarity("actor", "actress"))

print("Most similar words for 'boring':")
print(model.most_similar("boring",topn=15))


In [ ]:
import numpy
import os
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects

numpy.save("vectors_in.npy",model.vectors[:1000,:]) # save vectors for a file

# run tsne locally, input is an embedding matrix, output is an embedding matrix with reduced dimensions
os.system("python3 bhtsne_wrapper.py --input vectors_in.npy --output vectors_out.npy")

m2d=numpy.load("vectors_out.npy") # read new vectors from a file

# plot
fig, ax = plt.subplots(figsize=(50, 50), dpi=250)
plt.scatter(m2d[:1000,0],m2d[:1000,1],marker=",",color="gray")
words=[(k,w.index) for k,w in model.vocab.items()][:1000] # read words from the embedding model
words=[w for w,i in sorted(words, key=lambda x:x[1])] # sort based on the index to make sure the order is correct
print(words[:50])

for i,(v,w) in enumerate(zip(m2d,words)):
    if i%2!=0:
        continue
    txt=plt.text(v[0],v[1],w,size=30,ha="center",va="center",color="black")
    txt.set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'),
                   path_effects.Normal()])

plt.show()